In [ ]:
from huggingface_hub import notebook_login
import evaluate
notebook_login()

!pip install -q --upgrade datasets huggingface_hub fsspec
!pip install -U transformers
!pip install transformers datasets evaluate seqeval

import transformers, datasets, torch, accelerate, evaluate

from datasets import load_from_disk
ds = load_from_disk("../datasets/20_newsgroups_ds")
ds

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = ds.map(tokenize_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(200))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(200))

# Setup evaluation
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Load pretrained model and evaluate model after each epoch
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased" , num_labels=20)

from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir='model',
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

eval_results = trainer.evaluate()
print(eval_results)

from huggingface_hub import notebook_login
!huggingface-cli login --token TOKEN
!huggingface-cli login -h
!pip3 install git-lfs

repo_name = "lighthouse-NLP-assignment"
training_args = TrainingArguments(
   output_dir=repo_name,
   push_to_hub=True,
)

trainer.push_to_hub()

In [ ]:
# https://huggingface.co/mfaz3/model